In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('insert_your_data_path', one_hot=True)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Extracting /Users/urim/Data/MNIST_data/train-images-idx3-ubyte.gz
Extracting /Users/urim/Data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting /Users/urim/Data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting /Users/urim/Data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
hidden_size1 = 32
hidden_size2 = 64
batch_size = 100
lr = 0.001

In [3]:
inputs = tf.placeholder(tf.float32, shape=[None, 784]) #B,28*28
labels = tf.placeholder(tf.float32, shape=[None, 10]) #

inputs_reshape = tf.reshape(inputs, shape=[-1, 28, 28, 1]) # batchsize,28,28,channel

W1 = tf.Variable(tf.random_normal([3,3,1,hidden_size1]))
b1 = tf.Variable(tf.random_normal([256]))
conv1 = tf.nn.conv2d(inputs_reshape, W1, strides=[1,1,1,1], padding='SAME')
conv1_relu = tf.nn.relu(conv1)

pool1 = tf.nn.max_pool(conv1_relu, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') #size: [batchsize,14,14,256]

W2 = tf.Variable(tf.random_normal([3,3,hidden_size1,hidden_size2]))
b1 = tf.Variable(tf.random_normal([128]))
conv2 = tf.nn.conv2d(pool1, W2, strides=[1,1,1,1], padding='SAME')
conv2_relu = tf.nn.relu(conv2)

pool2 = tf.nn.max_pool(conv2_relu, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME') #size= [batchsize,7,7,128]
pool2_reshape = tf.reshape(pool2, shape=[-1, hidden_size2*7*7])

W3 = tf.Variable(tf.random_normal([hidden_size2*7*7,10]))
b3 = tf.Variable(tf.random_normal([10]))
fc1 = tf.matmul(pool2_reshape, W3) + b3

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=fc1, labels=labels))
train_step = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)

acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf.nn.softmax(fc1), axis=1), tf.argmax(labels, axis=1)), tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter('/Users/urim/Scripts/GIT', sess.graph)
    for step in range(500 + 1):
        batch_tr_inputs, batch_tr_labels = mnist.train.next_batch(batch_size)
        _, tr_loss, tr_acc = sess.run([train_step, loss, acc], 
                                      feed_dict={inputs: batch_tr_inputs, labels: batch_tr_labels})
        
        batch_val_inputs, batch_val_labels = mnist.validation.next_batch(batch_size)
        val_loss, val_acc = sess.run([loss, acc], 
                                      feed_dict={inputs: batch_val_inputs, labels: batch_val_labels})
        
        batch_te_inputs, batch_te_labels = mnist.test.next_batch(batch_size)
        te_loss, te_acc = sess.run([loss, acc], 
                                      feed_dict={inputs: batch_te_inputs, labels: batch_te_labels})
        
        if step % 100 == 0:
            print('step: {} \ntr_loss: {:0.4f} \ttr_acc: {:0.4f} \nval_loss: {:0.4f} \tval_acc: {:0.4f} \nte_loss: {:0.4f} \tte_acc: {:0.4f}\n'
                  .format(step, tr_loss, tr_acc, val_loss, val_acc, te_loss, te_acc))
        

step: 0 
tr_loss: 3618.5227 	tr_acc: 0.0700 
val_loss: 3378.3438 	val_acc: 0.0600 
te_loss: 2993.8081 	te_acc: 0.1000

step: 100 
tr_loss: 207.5100 	tr_acc: 0.7300 
val_loss: 256.2859 	val_acc: 0.7400 
te_loss: 227.5001 	te_acc: 0.7600

step: 200 
tr_loss: 205.0798 	tr_acc: 0.8200 
val_loss: 99.1273 	val_acc: 0.8700 
te_loss: 171.2691 	te_acc: 0.8100

step: 300 
tr_loss: 138.2458 	tr_acc: 0.8500 
val_loss: 55.9431 	val_acc: 0.9200 
te_loss: 75.8406 	te_acc: 0.8900

step: 400 
tr_loss: 51.0578 	tr_acc: 0.9200 
val_loss: 32.6542 	val_acc: 0.9300 
te_loss: 38.3189 	te_acc: 0.9300

step: 500 
tr_loss: 97.5805 	tr_acc: 0.8700 
val_loss: 39.7727 	val_acc: 0.8900 
te_loss: 76.4438 	te_acc: 0.9200

